In [1]:
import boto3
import json
import numpy as np

# --- CONFIGURATION (MUST MATCH DEPLOYMENT) ---
REGION = "us-east-1" # Must match the REGION in your deployment script
ENDPOINT_NAME = "netflix-revenue-endpoint-20251025-134503" # MUST match the unique name from your deployment output

# --- SAMPLE DATA (MUST MATCH TRAINING/INFERENCE SCRIPT) ---
# Your training script used these features: 
# ["subscribers", "avg_watch_hours", "monthly_fee", "marketing_spend"]
sample_data = {
    "subscribers": [5000000],
    "avg_watch_hours": [5.5],
    "monthly_fee": [15.99],
    "marketing_spend": [250000]
}

# SageMaker expects a list of records if it's a multi-row input, 
# or you might need to structure it exactly as your input_fn expects (which was pd.DataFrame(json.loads(request_body)))
# For a simple LinearRegression deployment using the standard Scikit-learn container, 
# often a list of lists or a specific JSON structure works best for input. 
# Let's format it as a single record matching the expected DataFrame structure when serialized:
input_payload = json.dumps(sample_data) 

# --- INVOCATION ---
try:
    # 1. Initialize the SageMaker Runtime Client
    runtime = boto3.client('sagemaker-runtime', region_name=REGION)
    
    # 2. Invoke the Endpoint
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType='application/json',  # Must match what your input_fn expects
        Accept='application/json',       # Desired output format
        Body=input_payload
    )
    
    # 3. Process the Response
    result = response['Body'].read().decode('utf-8')
    prediction = json.loads(result)
    
    print("✅ Successfully invoked endpoint!")
    print(f"Input Data: {sample_data}")
    print(f"Predicted Revenue: {prediction}")

except Exception as e:
    print(f"❌ Error during invocation: {e}")
    # If you get the UnknownOperationException here, it means the authentication 
    # for this specific script is wrong, or the endpoint name is misspelled.

✅ Successfully invoked endpoint!
Input Data: {'subscribers': [5000000], 'avg_watch_hours': [5.5], 'monthly_fee': [15.99], 'marketing_spend': [250000]}
Predicted Revenue: [79935495.3833963]
